In [46]:
import numpy as np
import scipy
from copy import deepcopy
import matplotlib.pyplot as plt

# data is stored at https://drive.google.com/drive/folders/1mH7xaQa3Hc2A5H0U5Ebd0EjLdltv1ojU

In [47]:
K = 50 # number of OFDM subcarriers
CP = K//2  # length of the cyclic prefix: 25% of the block
P = 25 # number of pilot carriers per OFDM block
pilotValue = (3+3j) # The known value each pilot transmits
allCarriers = np.arange(K)  # indices of all subcarriers ([0, 1, ... K-1])
pilotCarriers = allCarriers[::K//P] # Pilots is every (K/P)th carrier.
# For convenience of channel estimation, let's make the last carriers also be a pilot
pilotCarriers = np.hstack([pilotCarriers, np.array([allCarriers[-1]])])
P = P+1
# data carriers are all remaining carriers
dataCarriers = np.delete(allCarriers, pilotCarriers)

mu = 4 # bits per symbol (i.e. 16QAM)
payloadBits_per_OFDM = len(dataCarriers)*mu  # number of payload bits per OFDM symbol

mapping_table = {
    (0,0,0,0) : -3-3j,
    (0,0,0,1) : -3-1j,
    (0,0,1,0) : -3+3j,
    (0,0,1,1) : -3+1j,
    (0,1,0,0) : -1-3j,
    (0,1,0,1) : -1-1j,
    (0,1,1,0) : -1+3j,
    (0,1,1,1) : -1+1j,
    (1,0,0,0) :  3-3j,
    (1,0,0,1) :  3-1j,
    (1,0,1,0) :  3+3j,
    (1,0,1,1) :  3+1j,
    (1,1,0,0) :  1-3j,
    (1,1,0,1) :  1-1j,
    (1,1,1,0) :  1+3j,
    (1,1,1,1) :  1+1j
}

demapping_table = {v : k for k, v in mapping_table.items()}

In [48]:
def SP(bits):
    return bits.reshape((len(dataCarriers), mu))

def Mapping(bits):
    return np.array([mapping_table[tuple(b)] for b in bits])

def OFDM_symbol(QAM_payload):
    symbol = np.zeros(K, dtype=complex) # the overall K subcarriers
    symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers
    symbol[dataCarriers] = QAM_payload  # allocate the pilot subcarriers
    return symbol

def IDFT(OFDM_data):
    return np.fft.ifft(OFDM_data)

def addCP(OFDM_time):
    cp = OFDM_time[-CP:]               # take the last CP samples ...
    return np.hstack([cp, OFDM_time])  # ... and add them to the beginning

def channel(signal, H__delay):
    convolved = np.convolve(signal, H__delay)
    return convolved

def removeCP(signal):
    return signal[CP:(CP+K)]

def DFT(OFDM_RX):
    return np.fft.fft(OFDM_RX)

def channelEstimate_from_pilots(OFDM_demod):
    pilots = OFDM_demod[pilotCarriers]  # extract the pilot values from the RX signal
    Hest_at_pilots = pilots / pilotValue # divide by the transmitted pilot values

    # Perform interpolation between the pilot carriers to get an estimate
    # of the channel in the data carriers. Here, we interpolate absolute value and phase
    # separately
    Hest_abs = scipy.interpolate.interp1d(pilotCarriers, abs(Hest_at_pilots), kind='linear')(allCarriers)
    Hest_phase = np.angle(scipy.interpolate.interp1d(pilotCarriers, Hest_at_pilots, kind='linear')(allCarriers))
    Hest = Hest_abs * np.exp(1j*Hest_phase)
    return Hest

def equalize(OFDM_demod, Hest):
    return OFDM_demod / Hest

def get_payload(equalized):
    return equalized[dataCarriers]

def Demapping(QAM):
    # array of possible constellation points
    constellation = np.array([x for x in demapping_table.keys()])

    # calculate distance of each RX point to each possible point
    dists = abs(QAM.reshape((-1,1)) - constellation.reshape((1,-1)))

    # for each element in QAM, choose the index in constellation
    # that belongs to the nearest constellation point
    const_index = dists.argmin(axis=1)
    
    # get back the real constellation point
    hardDecision = constellation[const_index]

    # transform the constellation point into the bit groups
    return np.vstack([demapping_table[C] for C in hardDecision]), hardDecision

def PS(bits):
    return bits.reshape((-1,))

In [49]:
H__sample_freq = np.load('./data1d/test/H__sample_fu.npy')
H_noisy__sample_freq = np.load('./data1d/test/H_noisy__sample_fu.npy')

In [50]:
def get_denoised_channel(H_noisy__freq, H__freq):   
    #TODO: This is a very simple model, where the high-delay parts (30% highest) are considered noise and are simply removed.
    #TODO: Modify this function to use AI instead
    H_denoised__freq = deepcopy(H_noisy__freq)
    H_denoised__dct = scipy.fftpack.idct(H_denoised__freq)
    H_denoised__dct[-int(len(H_denoised__dct)*0.3):] = 0.
    H_denoised__freq = scipy.fftpack.dct(H_denoised__dct)
    return H_denoised__freq

def get_BER(sample):
    H__freq = H__sample_freq[sample]
    H_noisy__freq = H_noisy__sample_freq[sample]
    H_denoised__freq = get_denoised_channel(H_noisy__freq, H__freq)
    H__delay = IDFT(H__freq)
    H_noisy__delay = IDFT(H_noisy__freq)
    H_denoised__delay = IDFT(H_denoised__freq)

    # H_denoised__delay = get_denoised_channel(H_noisy__delay)
    bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM,))
    bits_SP = SP(bits)
    QAM = Mapping(bits_SP)
    OFDM_data = OFDM_symbol(QAM)
    OFDM_time = IDFT(OFDM_data)
    OFDM_withCP = addCP(OFDM_time)
    OFDM_TX = OFDM_withCP
    BERs = np.zeros((2,))
    for index, channel_response in enumerate([H_noisy__delay, H_denoised__delay]):
        OFDM_RX = channel(OFDM_TX, channel_response)            # This is not true in reality, but good enough for our toy example with only 50 subcarriers and 1 time slot.
        OFDM_RX_noCP = removeCP(OFDM_RX)
        OFDM_demod = DFT(OFDM_RX_noCP)
        Hest = channelEstimate_from_pilots(OFDM_demod)          # In reality, denoising should be done here instead.
        equalized_Hest = equalize(OFDM_demod, Hest)
        QAM_est = get_payload(equalized_Hest)
        PS_est, _ = Demapping(QAM_est)
        bits_est = PS(PS_est)
        BERs[index] = np.sum(abs(bits-bits_est))/len(bits)
    return BERs

nrof_sample = min(len(H__sample_freq), 10000)            # choose 
BER = np.ones((nrof_sample))
BER_noisy = np.ones((nrof_sample))
BER_denoised = np.ones((nrof_sample))

for sample in range(nrof_sample):
    BER_noisy[sample], BER_denoised[sample] = get_BER(sample)

print(f"Using {nrof_sample} 1Tx-1Rx pairs to find Bit Error Rate (BER).")
print(f"BER for noisy channel: {np.mean(BER_noisy):.3f}")
print(f"BER for denoised channel: {np.mean(BER_denoised):.3f}")

Using 10000 1Tx-1Rx pairs to find Bit Error Rate (BER).
BER for noisy channel: 0.189
BER for denoised channel: 0.115
